In [1]:
import pandas as pd
import numpy as np

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy.stats.mstats import winsorize
import statsmodels.api as sm
from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

# 데이터셋 로드
file_path = 'train_data.csv'
data = pd.read_csv(file_path)

# '회사명' 컬럼 제거
data = data.drop(['회사명'], axis=1)

# inf와 -inf를 NaN으로 변환
data.replace([np.inf, -np.inf], np.nan, inplace=True)

# NaN 값을 평균이나 중앙값으로 대체
data.fillna(data.mean(), inplace=True)

# 스탠다드 스케일링 적용
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data)

# 스케일링된 데이터를 데이터프레임으로 변환
scaled_data_df = pd.DataFrame(scaled_data, columns=data.columns)

# t-test와 라쏘 분석을 위한 데이터 준비
X = scaled_data_df.drop('label', axis=1)  # 'label' 컬럼을 제외한 모든 컬럼
y = scaled_data_df['label']

# t-test
X = sm.add_constant(X)  # 상수항 추가
model = sm.OLS(y, X).fit()
t_test_results = model.summary()

# 라쏘 분석
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
lasso = LassoCV(cv=5, random_state=0).fit(X_train, y_train)
lasso_coef = lasso.coef_

# 예측 및 RMSE 계산
y_pred = lasso.predict(X_test)
rmse = sqrt(mean_squared_error(y_test, y_pred))

t_test_results, lasso_coef, rmse


c:\Users\dlsgm\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.08373943119352134, tolerance: 0.07849559226430294
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\dlsgm\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.39413433272045495, tolerance: 0.07849559226430294
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\dlsgm\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.6700510807812634, tolerance: 0.07849559226430294
  model = cd_fast.enet_coordinate_descent

(<class 'statsmodels.iolib.summary.Summary'>
 """
                             OLS Regression Results                            
 Dep. Variable:                  label   R-squared:                       0.222
 Model:                            OLS   Adj. R-squared:                  0.190
 Method:                 Least Squares   F-statistic:                     6.904
 Date:                Thu, 30 Nov 2023   Prob (F-statistic):           5.81e-48
 Time:                        20:48:42   Log-Likelihood:                -2025.8
 No. Observations:                1566   AIC:                             4178.
 Df Residuals:                    1503   BIC:                             4515.
 Df Model:                          62                                         
 Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
 -------------------------------------------------------------